# 기본 패키지 다운로드

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,829 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 http

bash: line 5: fg: no job control


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1000k      0  0:00:01  0:00:01 --:--:-- 1740k
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

In [ ]:
! pip install konlpy

In [ ]:
import pandas as pd
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 데이터 전처리

## 데이터 로딩

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/리뷰마이닝/reviews_맞춤법 수정.csv')
data.iloc[793, -1] = 1

## 데이터 split

In [ ]:
X = data.iloc[:, 1];  y = data.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size = 0.3, shuffle = True, random_state = 42, \
                                                    stratify = y)
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
y_train = y_train.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

In [ ]:
X

0       선물로 줬는데 사용해보니 정말 만족스럽다고 하네요.
1                              추천합니다
2                            배송도 빠르고
3                       기사님도 친절하셨어요.
4                   디자인 너무너무 맘에 들고요.
                    ...             
1426        디자인도 공기청정기 중 가장 이쁜 거 같아요
1427                     디자인이 너무 예뻐요
1428              선물시켰는데 받는 이가 좋아하네요
1429                        배송이 늦었지만
1430             아주 고급스럽고 작동도 만족합니다.
Name: review, Length: 1431, dtype: object

## 데이터 형태소 분석

In [ ]:
mecab = Mecab()
train_raw_tokens_list = []; train_raw_tokens_append = train_raw_tokens_list.append
for index in range(len(X_train)):
    temp_comment = X_train[index]
    train_raw_tokens_append(mecab.pos(temp_comment))

In [ ]:
mecab = Mecab()
test_raw_tokens_list = []; test_raw_tokens_append = test_raw_tokens_list.append
for index in range(len(X_test)):
    temp_comment = X_test[index]
    test_raw_tokens_append(mecab.pos(temp_comment))

## 형태소 분석을 바탕으로 데이터 토큰화
+ 살릴 품사: 'NNG', 'VV', 'VA'

In [ ]:
train_clean_tokens_list = []; train_clean_tokens_append = train_clean_tokens_list.append
for i in range(len(train_raw_tokens_list)):
    temp_raw_tokens = train_raw_tokens_list[i]
    temp_clean_tokens = []
    for tup in temp_raw_tokens:
        if tup[1] in ['NNG', 'NNP']:
            temp_clean_tokens.append(tup[0])
    train_clean_tokens_append(temp_clean_tokens)

In [ ]:
test_clean_tokens_list = []; test_clean_tokens_append = test_clean_tokens_list.append
for i in range(len(test_raw_tokens_list)):
    temp_raw_tokens = test_raw_tokens_list[i]
    temp_clean_tokens = []
    for tup in temp_raw_tokens:
        if tup[1] in ['NNG', 'NNP']:
            temp_clean_tokens.append(tup[0])
    test_clean_tokens_append(temp_clean_tokens)

# 단어 데이터프레임

In [ ]:
word_tokens = []
for i in range(0, len(train_clean_tokens_list)):
    for j in range(0, len(train_clean_tokens_list[i])):
        word_tokens.append(train_clean_tokens_list[i][j])

In [ ]:
word_tokens = pd.DataFrame(word_tokens)

In [ ]:
word_tokens[word_tokens[0] == '에어'] = '에어로타워'

In [ ]:
word_tokensd = word_tokens[word_tokens[0]!='타워']

In [ ]:
word_tokens.rename(columns={0:'review'}, inplace=True)

In [ ]:
word_tokens

,review
0,온풍
1,전기료
2,걱정
3,크기
4,무게
...,...
2569,기능
2570,포함
2571,온풍
2572,전달


In [ ]:
# word_tokens.to_csv('word_tokens.csv', index = False, mode = 'w', encoding = 'utf-8-sig')

In [ ]:
word_tokens['review'].value_counts()

배송     114
온풍     101
디자인     92
공기      91
기능      88
      ... 
애        1
남        1
거사       1
이해       1
전달       1
Name: review, Length: 570, dtype: int64

In [ ]:
copy = word_tokens.copy()

In [ ]:
stop = ['약간', '최근', '딸', '밍', '부', '때', '후', '달', '친언니', '전', '부탁', '아들', '나중', '덕분', '위', 
        '대신', '조카', '상의', '남편', '플로', '면', '여기저기', '오늘', '하이', '플', '점', '큐', '감', '이곳저곳',
        '돌', '몰', '앞', '비람', '씽', '훈기', '행', '이후', '기', '셔', '두말', '입']

In [ ]:
copy = copy.dropna()

In [ ]:
copy = copy[(copy.review != '약간') & (copy.review != '최근') & (copy.review != '딸') & (copy.review != '밍') & (copy.review != '부') & (copy.review != '때') &
     (copy.review != '후') & (copy.review != '달') & (copy.review != '친언니') & (copy.review != '전') & (copy.review != '부탁') & (copy.review != '아들') &
      (copy.review != '나중') & (copy.review != '덕분') & (copy.review != '위') & (copy.review != '대신') & (copy.review != '조카') & (copy.review != '상의') &
      (copy.review != '남편') & (copy.review != '플로') & (copy.review != '면') & (copy.review != '여기저기') & (copy.review != '오늘') & (copy.review != '하이') &
     (copy.review != '플') & (copy.review != '점') & (copy.review != '큐') & (copy.review != '감') & (copy.review != '이곳저곳') & (copy.review != '돌') &
     (copy.review != '몰') & (copy.review != '앞') & (copy.review != '비람') & (copy.review != '씽') & (copy.review != '훈기') & (copy.review != '행') & (copy.review != '이후') &
     (copy.review != '기') & (copy.review != '셔') & (copy.review != '두말') & (copy.review != '입') & (copy.review != '애') & (copy.review != '남') & (copy.review != '거사') &
     (copy.review != '이해') & (copy.review != '전달') & (copy.review != '팬') & (copy.review != '기') & (copy.review != '다음') & (copy.review != '날') & (copy.review != '배') &
     (copy.review != '면') & (copy.review != '뒤') & (copy.review != '마왕') & (copy.review != '가요') & (copy.review != '신념') & (copy.review != '속') & (copy.review != '롯') &
     (copy.review != '이번') & (copy.review != '기회') & (copy.review != '신') & (copy.review != '차') & (copy.review != '선짝') & (copy.review != '안') & (copy.review != '눈') &
     (copy.review != '월') & (copy.review != '원') & (copy.review != '밑') & (copy.review != '동생') & (copy.review != '제주도') & (copy.review != '정도') & (copy.review != '옆') &
     (copy.review != '곳') & (copy.review != '예') & (copy.review != '딜') & (copy.review != '히') & (copy.review != '도') & (copy.review != '기와') & (copy.review != '막') &
     (copy.review != '댁') & (copy.review != '사') & (copy.review != '오른쪽') & (copy.review != '왼쪽') & (copy.review != '상도') & (copy.review != '요번') & (copy.review != '티브이') &
     (copy.review != '그간') & (copy.review != '키') & (copy.review != '철') & (copy.review != '라모') & (copy.review != '도') & (copy.review != '도') & (copy.review != '도') &
     (copy.review != '도') & (copy.review != '도') & (copy.review != '도') & (copy.review != '도') & (copy.review != '도') & (copy.review != '도') & (copy.review != '도') ]

In [ ]:
copy.review.unique()

array(['온풍', '전기료', '걱정', '크기', '무게', '실물', '디자인', '설치', '서비스', '후회', '집',
       '배송', '기사', '친절', '크림색', '화이트', '벽지', '기존', '퓨리', '케어', '공기',
       '청정기', '사용', '소음', '인테리어', '효과', '광택', '색감', '디스플레이', '밝기', '조절',
       '방', '불편', '겨울', '거실', '도움', '베이지', '색', '안내', '환기', '공청', '중요',
       '청정', '기능', '유용', '감사', '가전제품', '만족', '연결', '정화', '범위', '온도',
       '전기세', '각오', '장시간', '폭탄', '샤워', '몸', '바람', '협곡', '모티브', '침실', '컬러',
       '무드', '가격', '이사', '주문', '앱', '컨트롤', '아이', '날씨', '신혼', '선물', '온열',
       '선풍', '구매', '일정', '조율', '오브제', '에어로타워', '타워', '마음', '제품', '시간',
       '여름', '송풍', '가능', '기대', '생각', '사이즈', '회전각', '차이', '숙면', '안방', '고민',
       '가전', '아래', '순환', '사무실', '전체', '현상', '색상', '실내', '선풍기', '온풍기',
       '공간', '차지', '관리', '끝', '결혼', '설명', '핸드폰', '원격', '대체', '오프라인',
       '판매점', '롯데', '연상', '선선', '기분', '명절', '최고', '강점', '절정', '예정일', '문',
       '아침', '자체', '집안', '강추', '다운로드', '업그레이드', '외출', '외풍', '성능', '방비',
       '절약', '모드', '희망', '설정', '자동', '편리', '정상', '작동', '모양', '직

In [ ]:
copy

,review
0,온풍
1,전기료
2,걱정
3,크기
4,무게
...,...
2568,청정
2569,기능
2570,포함
2571,온풍


In [ ]:
for i in range(0, len(copy.review.value_counts())):
    if copy.review.value_counts()[i] < 2:
        print(copy.review.value_counts().index[i])

순간
대부분
감격
반영
예상
작업실
역할
건강
이용
센추리
최적
의견
조정
모던
고양이
계절
청소
열기
단계
행사
최초
저가
취침
임시
잠실
살균
비닐
여유
역세
일
요리
교체
공공
시기
아쉬움
롯데월드
기본
불안
작년
기간
면적
냉기
친정집
신세계
숨
출시
포장
새해
부피
모습
토끼
미니어처
흠집
세탁기
세분
유일
타사
평안
입주
값어치
침대
택배
분위기
주방
순차
사도
분실
스케줄
사전
예약
기술력
고개
짜증
기관지
진심
장점
오랫동안
계열
사진
자석
건조기
보통
알레르기
상승
신분
쇼핑몰
운동
믿음
컬렉션
출산
사육
실망
외부
거대
극찬
흡입
가습기
털
반려
그린
감동
비교
한수
환절기
감기
조심
냉장고
심플
염려
관련
목록
초미세
임신
비주얼
실제
제거
능력
퇴근
감지
경남
시스템
선호
제품력
신뢰도
신뢰
향기
아내
얼마
전원
라디에이터
전기난로
난리
문제
해결
흰색
안성맞춤
돈
인생
템
회전
반경
다양
완벽
관심
연동
건조
온열
무게
크림색
벽지
광택
색감
디스플레이
밝기
도움
범위
각오
폭탄
협곡
모티브
무드
조율
직원
회전각
차이
관리
결혼
대체
오프라인
판매점
연상
선선
강점
절정
예정일
외풍
정상
피부
부착
청
정확
출발
전날
내일
전환
개발자
시즌
각도
인증
과열
대박
진작
겸용
친정
주택
랜드라
시국
결정
한잔
리뷰
얘기
사은품
톤
컴퓨터
페인팅
생활
다용도
손
시선
졸음
공감
해피콜
코로나
아기
밤낮
반구
주의
아파트
아트
해당
판매
소비자
방문
천안
택시
만능
사고
방지
가드
도착
종일
가치
불만족
본가
미스트
기온
목감기
스마트폰
환풍기
소요
주전자
특유
어린아이
겨울철
자녀
포함


## 토큰화된 데이터 벡터화

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

train_final = []
test_final = []
for i in range(0, len(train_clean_tokens_list)):
    str_list = train_clean_tokens_list[i]
    result = listToString(str_list)
    train_final.append(result)

for i in range(0, len(test_clean_tokens_list)):
    str_list = test_clean_tokens_list[i]
    result = listToString(str_list)
    test_final.append(result)


In [ ]:
train_final = pd.Series(train_final)
test_final = pd.Series(test_final)

In [ ]:
train_final

0               온풍 전기료 걱정
1                크기 무게 실물
2                     디자인
3                  설치 서비스
4                      후회
              ...        
996     송풍 온풍 공기 청정 기능 포함
997                      
998                      
999                 온풍 전달
1000                   만족
Length: 1001, dtype: object

In [ ]:
test_final

0      퓨리 케어 에어 타워 오브제 컬렉션 글 렌 디자인 크림 화이트 컬러 화이트 인테리어...
1                                               자리 차지 만족
2                                                      맘
3                                                     수령
4                                                       
                             ...                        
425                                                   소리
426                                                   추천
427                                                친절 설치
428                                             회전 반경 다양
429                                                     
Length: 430, dtype: object

In [ ]:
tfv = TfidfVectorizer()
tfv_X_train = tfv.fit_transform(train_final)
tfv_X_test = tfv.transform(test_final)

## 모델링

### 모델 학습

In [ ]:
clf = LogisticRegression(random_state = 42)
clf.fit(tfv_X_train, y_train)

LogisticRegression(random_state=42)

### 예측

In [ ]:
tfv_X_test = tfv.transform(X_test)

### 모델 정확도

In [ ]:
def cal_acc(y_pred, y_true):
    N = len(y_pred)
    return (sum(y_pred == y_true) / N)

In [ ]:
cal_acc(clf.predict(tfv_X_test), y_test)

0.8209302325581396

In [ ]:
len(test_clean_tokens_list)

430

In [ ]:
clean_tokens_list = train_clean_tokens_list + test_clean_tokens_list
clean_tokens_df = pd.DataFrame(data = None, index = range(1431), columns = ['tokens'])
clean_tokens_df

,tokens
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1426,NaN
1427,NaN
1428,NaN
1429,NaN


In [ ]:
for index in clean_tokens_df.index:
    clean_tokens_df.iloc[index, 0] = clean_tokens_list[index]

In [ ]:
clean_tokens_df.to_csv('clean_tokens_df.csv', index = False, mode = 'w', encoding = 'utf-8-sig')

In [ ]:
clean = train_clean_tokens_list.copy() + test_clean_tokens_list.copy()

In [ ]:
import pickle
my_list = clean
 
## Save pickle
with open("data.pickle","wb") as fw:
    pickle.dump(my_list, fw)
